In [1]:
import discord
import os
from dotenv import load_dotenv

from api_keys import *
from command_handling import *

from DESTINY import basics
from DESTINY import manifestation
from DESTINY import hbfunc

from selenium import webdriver

import bs4
import urllib

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

import json
import requests
import shutil

import numpy as np

In [2]:
all_data = manifestation.get_all_data(regen=False)

Not regenerating manifest


In [3]:
weapon_dict = manifestation.get_weapon_dict(all_data)

In [4]:
perk_dict = manifestation.get_perk_dict(all_data)

In [5]:
for ep in all_data.keys():
    print(ep)

DestinyPlaceDefinition
DestinyActivityDefinition
DestinyActivityTypeDefinition
DestinyClassDefinition
DestinyGenderDefinition
DestinyInventoryBucketDefinition
DestinyRaceDefinition
DestinyTalentGridDefinition
DestinyUnlockDefinition
DestinyStatGroupDefinition
DestinyFactionDefinition
DestinyVendorGroupDefinition
DestinyRewardSourceDefinition
DestinyItemCategoryDefinition
DestinyDamageTypeDefinition
DestinyActivityModeDefinition
DestinyMedalTierDefinition
DestinyAchievementDefinition
DestinyActivityGraphDefinition
DestinyBondDefinition
DestinyCollectibleDefinition
DestinyDestinationDefinition
DestinyEquipmentSlotDefinition
DestinyEventCardDefinition
DestinyStatDefinition
DestinyInventoryItemDefinition
DestinyItemTierTypeDefinition
DestinyLoadoutColorDefinition
DestinyLoadoutIconDefinition
DestinyLoadoutNameDefinition
DestinyLocationDefinition
DestinyLoreDefinition
DestinyMaterialRequirementSetDefinition
DestinyMetricDefinition
DestinyObjectiveDefinition
DestinySandboxPerkDefinition
Dest

In [6]:
def find_id(search_id):
    for key in all_data.keys():
        if search_id in all_data[key].keys():
            print(key, "\n", json.dumps(all_data[key][search_id], indent=4))

###  Statmax/statmin command:
similar to sametype, but find the one with the highest/lowest stat in a given category

In [7]:
import difflib

Pseudocode for  statmin/statmax:  
ah idk if I wanna bother with types and archetypes ehhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh


In [8]:
def remove_outer_spaces(s:str):
    """
    Remove leading or trailing spaces from string s and  return
    """
    mid_s = ""
    
    # handling leading
    leading=True
    for c in s:
        if leading is True:
            if c == " ":
                continue
            else:
                leading=False
                mid_s+=c
        
        else:
            mid_s += c
            
    final_s_r = ""
    
    # handling trailing
    n = len(mid_s)
    trailing=True
    for i in range(len(mid_s)):
        i_r = n - i - 1
        
        c_r  = mid_s[i_r]
        if trailing is True:
            if c_r == " ":
                continue
            else:
                final_s_r+=c_r
                trailing=False
                
        else:
            final_s_r+=c_r
    
    # reverse
    final_s = ""
    for i in range(len(final_s_r)):
        i_r = len(final_s_r) - i - 1
        c_r = final_s_r[i_r]
        final_s += c_r
    
    
    return final_s


remove_outer_spaces("    ab cde ")

'ab cde'

In [9]:
def statorder(message, args, client, all_data, weapon_dict):
    # need to also specify desired stat, which would come from args...
    weap, stat = " ".join(args).split("&")
    weap = remove_outer_spaces(weap)
    stat = remove_outer_spaces(stat)
    print(weap, stat)
    sametype_string = hbfunc.sametype(message, args, client, all_data, weapon_dict)
    
    for i, line in enumerate(sametype_string.split("\n")):
        # skip the first one
        if i ==0:
            continue
            
        # skip line breaks
        if len(line)==0:
            continue
            
        # if you reach these tiers, end the loop
        if line in ["**RARE**", "**UNCOMMON**",  "**COMMON**"]:
            break
        
        # if it's a title or 
        elif line in ["**EXOTIC**", "**LEGENDARY**"]:
            continue
            
        else:
            # this is an Exotic or Legendary weapon name
            print(line)
            print(hbfunc.weapstat(message, line.split(" "), client, all_data, weapon_dict))
    

statorder(None, ["The", "Palindrome", "&", "zoom"], None, all_data, weapon_dict)

The Palindrome zoom
Ace of Spades
Name: Ace of Spades
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 75
Stability: 40
Magazine: 13
Reload Speed: 42
Handling: 40
Aim Assistance: 70
Recoil Direction: 100
Airborne Effectiveness: 14

Hawkmoon
Name: Hawkmoon
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 52
Stability: 63
Magazine: 8
Reload Speed: 59
Handling: 71
Aim Assistance: 93
Recoil Direction: 96
Airborne Effectiveness: 26

Lumina
Name: Lumina
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 53
Stability: 50
Magazine: 13
Reload Speed: 61
Handling: 71
Aim Assistance: 88
Recoil Direction: 95
Airborne Effectiveness: 27

Thorn
Name: Thorn
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 34
Stability: 55
Magazine: 11
Reload Speed: 40
Handling: 65
Aim Assistance: 85
Recoil Direction: 100
Airborne Effectiveness: 21

Ancient Gospel
Name: Ancient Gospel
Type: Legendary Hand Cannon
Zoom: 1

In [10]:
print(hbfunc.weapstat(None, ["Fatebringer","(Timelost)"], None, all_data, weapon_dict))

Name: Fatebringer (Timelost)
Type: Legendary Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 46
Stability: 57
Magazine: 10
Reload Speed: 46
Handling: 55
Aim Assistance: 84
Recoil Direction: 98



## Perk combo function
$combo Demolitionist & Adrenaline Junkie --> return weapons that can roll both *in different slots*

In [11]:
from DESTINY.hbfunc import get_closest_gun
#from DESTINY.hbfunc import rolls
import helpers

In [13]:
import inspect  # get function source code
#lines = inspect.getsource(rolls)  
#print(lines)

In [58]:
def rolls_(message, args, client, all_data, weapon_dict, retstr=True):
    """
    get all possible perk rolls for a given weapon
    """

    #weapon = " ".join(args)
    weapon = get_closest_gun(message, args, client, all_data, weapon_dict)[0]

    repeats = 1
    rolls = {}

    # some weapons (raid, exotic, etc) have a different index for rolls
    # breaks for some other weapons...
    
    if len(weapon_dict[weapon]) == 3:
        idx = 1
    elif len(weapon_dict[weapon]) == 4:
        idx = 2
    else:
        idx = 0
    

    weaptype = weapon_dict[weapon][idx]['itemTypeAndTierDisplayName']
    
    print(weaptype)
    
    # for socket (usually barrels, mags, perk1, perk2, etc)
    for socket in weapon_dict[weapon][idx]['sockets']['socketEntries']:
        # if random rolls
        if "randomizedPlugSetHash" in socket.keys():
            randhash = socket['randomizedPlugSetHash']
            sockethash = socket['socketTypeHash']
            socketname = all_data['DestinySocketTypeDefinition'][sockethash]['plugWhitelist'][0]['categoryIdentifier']
        
            # for every perk that goes in that socket for this weapon
            perks = set()
            for perk in all_data['DestinyPlugSetDefinition'][randhash]['reusablePlugItems']:
                itemhash = perk['plugItemHash']
                perks.add(all_data['DestinyInventoryItemDefinition'][itemhash]['displayProperties']['name'])
                
            # save list of perks for each socket name
            if socketname == "frames":
                socketname = "Main Perk " + str(repeats)
                repeats += 1
            else:
                socketname = socketname[0].upper() + socketname[1:]
                
            rolls[socketname] = list(perks)


    if retstr is True:
        return(rolls_format(rolls, weapon, weaptype))

    return rolls # can specify retstr = False in parameters if you just want the dict back.

In [59]:
rolls_(None, ["Finite", "Impactor"], None, all_data, weapon_dict, retstr=False) 

Legendary Hand Cannon


{'Scopes': ['TrueSight HCS',
  'Sureshot HCS',
  'Crossfire HCS',
  'SteadyHand HCS',
  'Fastdraw HCS',
  'HitMark HCS'],
 'Magazines': ['Alloy Magazine',
  'Ricochet Rounds',
  'High-Caliber Rounds',
  'Extended Mag',
  'Appended Mag',
  'Armor-Piercing Rounds',
  'Light Mag',
  'Flared Magwell'],
 'Main Perk 1': ['Under Pressure',
  'Compulsive Reloader',
  'Slideways',
  'No Distractions',
  'Firmly Planted',
  'Stats for All',
  'Killing Wind',
  'Heating Up'],
 'Main Perk 2': ['Elemental Capacitor',
  'Multikill Clip',
  'Eye of the Storm',
  'Iron Reach',
  'Focused Fury',
  'Iron Grip',
  'One for All'],
 'Origins': ['Skulking Wolf', 'Suros Synergy']}

In [60]:
rolls_(None, ["Age-Old" ,"Bond"], None, all_data, weapon_dict, retstr=False) 

Legendary Auto Rifle


{'Barrels': ['Extended Barrel',
  'Hammer-Forged Rifling',
  'Chambered Compensator',
  'Full Bore',
  'Polygonal Rifling',
  'Corkscrew Rifling',
  'Arrowhead Brake',
  'Fluted Barrel',
  'Smallbore'],
 'Magazines': ['Alloy Magazine',
  'Extended Mag',
  'Tactical Mag',
  'Appended Mag',
  'Steady Rounds',
  'Accurized Rounds',
  'Flared Magwell'],
 'Main Perk 1': ['Stats for All',
  'Dynamic Sway Reduction',
  'Repulsor Brace',
  'Tap the Trigger',
  "Fourth Time's the Charm",
  'Dragonfly',
  'Discord'],
 'Main Perk 2': ['Golden Tricorn',
  'Golden Tricorn Enhanced',
  'Destabilizing Rounds',
  'Kill Clip',
  'Collective Action',
  'Focused Fury',
  'Target Lock',
  'Harmony']}

In [61]:
rolls_(None, ["Heritage"], None, all_data, weapon_dict, retstr=False) 

Legendary Shotgun


{'Barrels': ['Extended Barrel',
  'Hammer-Forged Rifling',
  'Chambered Compensator',
  'Full Bore',
  'Polygonal Rifling',
  'Corkscrew Rifling',
  'Arrowhead Brake',
  'Fluted Barrel',
  'Smallbore'],
 'Magazines': ['Extended Mag',
  'Assault Mag',
  'Tactical Mag',
  'Appended Mag',
  'Steady Rounds',
  'Light Mag',
  'Accurized Rounds'],
 'Main Perk 1': ['Dual Loader',
  'Pugilist',
  'Slideshot',
  'Auto-Loading Holster',
  'Demolitionist',
  'Hip-Fire Grip',
  'Outlaw',
  'Threat Detector',
  'Reconstruction'],
 'Main Perk 2': ['Moving Target',
  'Unrelenting',
  'Focused Fury',
  'Thresh',
  'Offhand Strike',
  'Recombination',
  'Killing Wind',
  'Swashbuckler',
  'Snapshot Sights'],
 'Origins': ['Bray Inheritance']}

In [66]:
lens = []
for w in weapon_dict.keys():
    ln = len(weapon_dict[w])
    lens.append(ln)
    
max(lens)

4